# Creating Coastal Wetlands Forests Habitat Suitability Dataset 


# Imports

In [2]:
# pip install earthengine-api==0.1.357

In [1]:
from dataset_builder import EEDatasetBuilder
import geemap
import ee

# Map

In [2]:
Map = geemap.Map(center=[27.295, -81.368], zoom=7)
Map

Map(center=[27.295, -81.368], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(chil…

# Dataset builder

In [3]:
# Build ee dataset builder
ee_dataset_builder = EEDatasetBuilder()

In [4]:
# C-CAP (class 13 with mangroves masked out)
subset_ccap = ee.Image('projects/wetlands-lab/assets/ccap_class_13_all_US_masked_mangroves') 

In [5]:
ee_dataset_builder.filtered_response_layer_from_raster(
    response_raster='custom', 
    ee_image=subset_ccap,
    custom_response_raster_name='response'
)

### Predictors

In [6]:
# Elevation - MERIT DEM
# Elevation and slope, which can be derived, should be core variables in determining where CFWs form.
dem = ee.Image('MERIT/DEM/v1_0_3');
predictors = dem.rename('elevation')

# Calculate slope in degrees
slope = ee.Terrain.slope(dem).rename('slope'); # fix this ? why is it not in degrees?
predictors = predictors.addBands(slope)

# Hydrology - MERIT HYDRO 
# Distance to water features can be determined with this data. CFW are likely to be close to water features.
merit_water = ee.Image('MERIT/Hydro/v1_0_1');
# Compute distance to water
water_mask = merit_water.select("viswth");
distance_to_water = water_mask.distance(ee.Kernel.euclidean(1000, 'meters')).rename('distance_water')
predictors = predictors.addBands(distance_to_water)

# JRC Global Surface Water
# Distance measures can be leveraged to understand proximity to water.
global_surface_water = ee.Image('JRC/GSW1_4/GlobalSurfaceWater');
# Compute distance to water
water_mask_global = global_surface_water.select("occurrence");
mask = water_mask_global.neq(0).rename('mask');
# Apply the mask to the image.
masked = water_mask_global.updateMask(mask);
# Set all masked values to 1.
water_mask_global_binary = masked.where(mask, 1).rename('global_water').unmask()
proximity_to_water = water_mask_global_binary.distance(ee.Kernel.euclidean(7000, 'meters'))
proximity_to_water = proximity_to_water.unmask().rename('proximity_water')
predictors = predictors.addBands(proximity_to_water)

# Soil moisture - Enhanced SMAP
# Wetlands tend to have much high soil moisture content compared to surrounding habitats.
dataset = ee.ImageCollection('NASA_USDA/HSL/SMAP10KM_soil_moisture').filter(ee.Filter.date('2016-01-01', '2016-12-30'));
soilMoisture = dataset.select('ssm').mean().rename('soil_moisture_mean');
predictors = predictors.addBands(soilMoisture) 

# # climate - Daymet
# # The various parameters can be summarized to determine the climate patterns.
# daymet = ee.ImageCollection('NASA/ORNL/DAYMET_V4').filter(ee.Filter.date('2016-01-01', '2016-12-30'));
# maximumTemperature = daymet.select('tmax').mean().rename('tmax_mean');
# minTemperature = daymet.select('tmin').mean().rename('tmin_mean');
# precipitation = daymet.select('prcp').mean().rename('precipitation_mean');
# predictors = predictors.addBands(maximumTemperature) 
# predictors = predictors.addBands(minTemperature) 
# predictors = predictors.addBands(precipitation) 

# Climate: NEX-GDDP-CMIP6: NASA Earth Exchange Global Daily Downscaled Climate Projections
nex_gppd_cmip6 = ee.ImageCollection("NASA/GDDP-CMIP6").filter(ee.Filter.date('2016-01-01', '2016-12-30')).filter(ee.Filter.eq('model', 'ACCESS-CM2'));
maximumTemperature = nex_gppd_cmip6.select('tasmax').mean().rename('tmax_mean');
minTemperature = nex_gppd_cmip6.select('tasmin').mean().rename('tmin_mean');
precipitation = nex_gppd_cmip6.select('pr').mean().rename('precipitation_mean');
predictors = predictors.addBands(maximumTemperature) 
predictors = predictors.addBands(minTemperature) 
predictors = predictors.addBands(precipitation) 


# Litology, Landforms, Physiography – Physical descriptions of the landscape, which can be used to understand landscape context.
# Soil lithology
lithology = ee.Image('CSP/ERGo/1_0/US/lithology').rename('lithology');
predictors = predictors.addBands(lithology)

# Topography: Landforms
landforms = ee.Image('CSP/ERGo/1_0/US/landforms');
landforms = landforms.select('constant').rename('landforms');
predictors = predictors.addBands(landforms)

# Soil and topography
physiography = ee.Image('CSP/ERGo/1_0/US/physiography');
physiography = physiography.select('constant').rename('physiography');
predictors = predictors.addBands(physiography)

Add predictors bands

In [7]:
predictors.bandNames().getInfo()

['elevation',
 'slope',
 'distance_water',
 'proximity_water',
 'soil_moisture_mean',
 'tmax_mean',
 'tmin_mean',
 'precipitation_mean',
 'lithology',
 'landforms',
 'physiography']

In [8]:
name_custom_ee_images_list = predictors.bandNames().getInfo()
ee_images_list = [predictors.select(band_name) for band_name in name_custom_ee_images_list]
predictors_list = ['custom_ee_image' for x in name_custom_ee_images_list]

In [9]:
ee_dataset_builder.spatial_covariates(covariates=predictors_list, 
                                      ee_image=ee_images_list,
                                      name_custom_ee_image=name_custom_ee_images_list ) 

List band names of image

In [10]:
ee_dataset_builder.image.bandNames().getInfo()

['response',
 'elevation',
 'slope',
 'distance_water',
 'proximity_water',
 'soil_moisture_mean',
 'tmax_mean',
 'tmin_mean',
 'precipitation_mean',
 'lithology',
 'landforms',
 'physiography']

### Visualize in map

In [11]:
vis_param = { 
  'min': 0.0,
  'max': 1.0,
  'palette':  [
    'white', 'blue'
  ]}

Map.addLayer(ee_dataset_builder.image.select('response'), vis_param, 'CCAP 13 + 16 bands masked mangroves', True)

In [14]:
elevation_vis_param = { 
  'min': 240,
  'max': 310,
  'palette': ['blue', 'purple', 'cyan', 'green', 'yellow', 'red'],
    'format': 'png'};

Map.addLayer(ee_dataset_builder.image.select('tmin_mean'), elevation_vis_param, 'minTemperature', True)

In [13]:
elevation_vis_param = { 
  'min': -3,
  'max': 18.0,
  'palette':  ['000000', '478FCD', '86C58E', 'AFC35E', '8F7131',
          'B78D4F', 'E2B8A6', 'FFFFFF'],
    'format': 'png'};

Map.addLayer(ee_dataset_builder.image.select('elevation'), elevation_vis_param, 'elevation', True)

In [15]:
Map.addLayer(ee_dataset_builder.image.select('proximity_water'), {'min': 0, 'max': 1000.0, 'palette':  ['white', 'blue']}, 'Proximity to Global Surface Water 100 meters kernel');


# Export samples CSV to GCP

In [16]:
# Gridded shapefile asset in GEE
# CCAP west coast 
shp_asset_path = 'projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_west_coast'

# CCAP south east coast
# shp_asset_path = 'projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_south_east_coast' 

 # CCAP north east coast
# shp_asset_path = 'projects/wetlands-lab/assets/ccap_mapping_bndry_gridded_50km_50km_north_east_coast' 

scale = 30
maxPixels = 1e13
gcp_bucket = 'wetlands-lab'
gcp_folder_name = 'habitat_suitability'
numPoints = 1000 # we override the number of points below with classPoints
classBand = "response"
classPoints = [4000, 1000]
classValues = [0,1]



In [17]:
samples_folder_name = f'west_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'
# samples_folder_name = f'south_east_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'
# samples_folder_name = f'north_east_coast_ccap_mapping_bndry_gridded_50km_50km_classPoints_4000_1000_classes_13_CCAP_new_climate_data'

# Stratified Sampling
# This will take quite some time
ee_dataset_builder.samples_csv_export(shp_asset_path=shp_asset_path, 
                                      name_gcp_bucket=gcp_bucket, 
                                      folder_in_gcp_bucket=gcp_folder_name + '/' + samples_folder_name, 
                                      scale=scale, 
                                      geometries=True,
                                      isStratifiedSampling=True, 
                                      numPoints=numPoints, 
                                      classValues=classValues,
                                      classBand=classBand, 
                                      classPoints=classPoints)

Geometry number of features: 402
Stratified sampling: 
numPoints: 1000, 
classBand: response, 
scale: 30, 
geometries: True, 
dropNulls: True, 
tileScale: 1, 
classPoints: [4000, 1000], 
seed: 0, 
projection:None


100%|████████████████████████████████████████████████████████████████████| 402/402 [1:16:06<00:00, 11.36s/it]


# Export tiles to GCP

In [ ]:
################## Export inference tiles to GCP bucket ##################
tiles_folder_name = f'tiles_west_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'
# tiles_folder_name = f'tiles_south_east_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'
# tiles_folder_name = f'tiles_north_east_coast_ccap_mapping_bndry_gridded_50km_50km_classes_13_CCAP_scale{scale}_new_climate_data'

print(f'\nExport inference tiles using the shapefile: {shp_asset_path}...')
# This will take quite some time
ee_dataset_builder.tiles_export(shp_asset_path,
                                name_gcp_bucket=gcp_bucket,
                                folder_in_gcp_bucket=gcp_folder_name + '/' + tiles_folder_name,
                                maxPixels=maxPixels, scale=scale)
print('Inference tiles export done.\n')